In [1]:

!pip install bitsandbytes
!pip install flash_attn
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.9 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.3.6-cp310-cp310-linux_x86_64.whl size=56598219 sha256=07a5c68948f0dbdc00e4ca74e7247ceb0c4adcde7f29d3de358c8ac2b8848709
  Stored in directory: /root/.cache/pip/wheels/24/5f/16/5044cdddb6dfb3331dfbffa28ab6096ec2900777af5cb0253a
Successfully built flash_attn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [3]:
model_name = 'LLM360/Amber'
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=quantization_config
                                             )


config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

pytorch_model-00002-of-00006.bin:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

pytorch_model-00003-of-00006.bin:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

pytorch_model-00004-of-00006.bin:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

pytorch_model-00005-of-00006.bin:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [5]:
token_ids = tokenizer("356 + 45 = ", return_tensors="pt").to("cuda")

In [6]:
tokens = tokenizer.convert_ids_to_tokens(token_ids["input_ids"][0])


In [7]:
tokens

['<s>', '▁', '3', '5', '6', '▁+', '▁', '4', '5', '▁=', '▁']

In [20]:
import numpy as np
inputs = tokenizer("Toronto is", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=15, return_dict_in_generate=True, output_scores=True)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)

output_scores = outputs["scores"]
scores_tensor = output_scores[1]
sorted_indices = torch.argsort(scores_tensor[0], descending=True)[:20]
for index in sorted_indices:
 token_id = index
 token_name = tokenizer.decode([token_id.item()])
 token_score = scores_tensor[0][index].item()
 print(f"Token: {token_name}, Score: {token_score}")

Token: city, Score: -147.875
Token: great, Score: -148.625
Token: very, Score: -149.25
Token: beautiful, Score: -149.375
Token: Canadian, Score: -149.375
Token: , Score: -149.5
Token: major, Score: -149.75
Token: good, Score: -149.875
Token: world, Score: -150.0
Token: wonderful, Score: -150.0
Token: v, Score: -150.125
Token: place, Score: -150.125
Token: large, Score: -150.125
Token: big, Score: -150.125
Token: global, Score: -150.25
Token: popular, Score: -150.25
Token: fun, Score: -150.5
Token: bit, Score: -150.5
Token: fant, Score: -150.5
Token: huge, Score: -150.625


In [22]:
#Reference: https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075
input_length = inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().numpy():.4f} | {np.exp(score.cpu().numpy()):.2%}")


|   263 | a        | -2.1565 | 11.57%
|  4272 | city     | -2.4637 | 8.51%
|   297 | in       | -1.6877 | 18.49%
| 21718 | Ontario  | -0.9541 | 38.52%
| 29892 | ,        | -0.3152 | 72.96%
|  7400 | Canada   | -0.0861 | 91.75%
| 29889 | .        | -0.5931 | 55.26%
|   739 | It       | -0.9860 | 37.31%
|   338 | is       | -0.3999 | 67.04%
|   278 | the      | -0.9951 | 36.97%
|  1556 | most     | -1.5086 | 22.12%
| 14938 | popul    | -0.2127 | 80.84%
|   681 | ous      | -0.0066 | 99.34%
|  4272 | city     | -0.4572 | 63.31%
|   297 | in       | -0.0341 | 96.65%


In [ ]:
del model
import gc
gc.collect()
